In [1]:
import pycutest
import numpy as np
import time
import pandas as pd

testes = [
    "ARGLINA", "ARGLINB", "BA-L1SPLS", "BIGGS6", "BROWNAL", "COATING",
    "FLETCHCR", "GAUSS2LS", "GENROSE", "HAHN1LS", "HEART6LS", "HILBERTB",
    "HYDCAR6LS", "LANCZOS1LS", "LANCZOS2LS", "LRIJCNN1", "LUKSAN12LS",
    "LUKSAN16LS", "OSBORNEA", "PALMER1C", "PALMER3C", "PENALTY2", "PENALTY3",
    "QING", "ROSENBR", "STRTCHDV", "TESTQUAD", "THURBERLS", "TRIGON1",
    "TOINTGOR"
]

In [2]:
class parameters:
    def __init__(self,function: str):
        self.function = pycutest.import_problem(function)                #deixar so os metodos publicos depois
        self.xk = self.function.x0                                      
        self.old_xk = None                                
        self.grad = self.function.grad(self.xk)                          #sempre teremos uma funcao gradiente para computar
        self.grad_calls = 1                                              #por isso inicializamos a chamada de gradiente como 1
        self.val_calls = 0
        self.old_grad = None
        #-------------------------------Caso Espectral primeiro passo------------------------
        self.get_new_point(self.xk - self.gradient())

        
    def get_new_point(self,x):
        self.old_xk = self.xk
        self.xk = x
        self.old_grad = self.grad
        self.grad = self.function.grad(self.xk)
        self.grad_calls += 1
        
    def gradient(self):
        return self.grad
    
    def objective(self, ponto):
        self.val_calls += 1
        return self.function.obj(ponto)

    def variation(self):         #retorna delta de x e delta de y
        return  (self.xk - self.old_xk ,self.grad - self.old_grad)  


In [3]:
def sigma(function: parameters)-> float:
    deltax, deltay = function.variation()
    """
    a partir de agora e feito o quadrados minimos
    de uma unica variavel
    com a aproximação da matriz quasi newton
    """
    skyk = np.dot(deltax,deltay)
    if skyk > 0.0:                                      
        sigma = skyk / np.dot(deltax,deltax)
    else: 
        sigma = 1.0e-4 * np.linalg.norm(function.grad, ord= np.inf) / max(1.0, np.linalg.norm(function.xk, ord = np.inf))
    return max(1.0e-30, min(sigma, 1.0e30))

def minimize(function: parameters, tolerance = 1e-5):            
    maximum_iterations = int(1e5)
    start = time.process_time()                                         
    for iteration in range(maximum_iterations):
        if np.linalg.norm(function.gradient(), ord= np.inf) < tolerance:
            end = time.process_time()
            tempo = end - start
            return {
                "val_calls": function.val_calls,
                "grad_calls": function.grad_calls,
                "tempo": tempo
            }
        else:
            dk = -function.gradient()/sigma(function)
            step = 1
            function.get_new_point(function.xk + step*dk)
    # Retorna negativo para indicar falha, se não convergir em máximo de iterações
    return {
        "val_calls": -function.val_calls,
        "grad_calls": -function.grad_calls,
        "tempo": 0.0
    }

In [4]:
espc = {i: minimize(parameters(i)) for i in testes}
df = pd.DataFrame(espc).T
df

/tmp/ipykernel_4599/2807489158.py:10: RuntimeWarning: invalid value encountered in scalar divide
  sigma = skyk / np.dot(deltax,deltax)
/tmp/ipykernel_4599/2807489158.py:30: RuntimeWarning: invalid value encountered in add
  function.get_new_point(function.xk + step*dk)
/tmp/ipykernel_4599/2807489158.py:12: RuntimeWarning: invalid value encountered in scalar divide
  sigma = 1.0e-4 * np.linalg.norm(function.grad, ord= np.inf) / max(1.0, np.linalg.norm(function.xk, ord = np.inf))


,val_calls,grad_calls,tempo
ARGLINA,0.0,3.0,0.003256
ARGLINB,0.0,44.0,0.112139
BA-L1SPLS,0.0,-100002.0,0.000000
BIGGS6,0.0,9.0,0.000157
BROWNAL,0.0,-100002.0,0.000000
COATING,0.0,-100002.0,0.000000
FLETCHCR,0.0,-100002.0,0.000000
GAUSS2LS,0.0,-100002.0,0.000000
GENROSE,0.0,37592.0,4.172937
HAHN1LS,0.0,2.0,0.000016


In [6]:
df.to_csv(r"Espectral_const_tabela2.csv",index=testes)